In [1]:
# DON'T CHANGE this part: import libraries
import numpy as np
import scipy
import json
from nltk.stem import PorterStemmer 
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 
import re
import itertools

In [2]:
# DON'T CHANGE this part: read data path
train_set_path, valid_set_path, random_number = input().split()

train.json valid.json 20


TODO:
1. preprocess: converting text to lowercase, coverting number, tokenization, removing stopword, stemming
2. embedding: hitogram matrix
3. classifier using linear regression
4. accuracy (for metric)

In [3]:
def loadData(train_set_path,valid_set_path):
    with open(train_set_path,"r") as file_train:
        training_data=json.load(file_train)
    file_train.close()

    with open(valid_set_path,"r") as file_valid:
        valid_data=json.load(file_valid)
    file_valid.close()
    return training_data,valid_data

#### Ví dụ cho phần báo cáo, nên báo cáo cho từng phần code để rõ ràng

Báo cáo phần tiền xử lý: dùng xyz để tách từ, ...

... Đối với những từ out-of-vocab (xuất hiện trong tập train nhưng không có ở tập valid), xử lý bằng cách ... 

In [4]:
# Ví dụ cho phần code hàm tiền xử lý

def preprocess(text):
    # converting text to lowercase
    text = text.lower()
    
    #text=re.sub("[^A-Za-z0-9]"," ",text)
    text=re.sub("[+-]?(\d+(\.\d+)?|\.\d+)([eE][+-]?\d+)?","num",text)
    """
    for word in text.split():
        if word.isdigit():
            text=re.sub(word,"num",text)
    """
    words=word_tokenize(text)
    
    stopword=stopwords.words("english")
    tmp=[word for word in words if word not in stopword]
    
    stemmer=PorterStemmer()
    words=[stemmer.stem(word) for word in tmp]
    return words

In [5]:
def preprocessed_training_data(training_data):
    preprocessed_text=list()
    labels=list()
    for data in training_data:
        preprocessed_text.append(preprocess(data["reviewText"]))
        labels.append(data["overall"])
    return preprocessed_text,labels

In [6]:
def buildGrossary(training_preprocessed_text):
    vocabulary=list()
    for preprocessed_text in training_preprocessed_text:
        for word in preprocessed_text:
            if word not in vocabulary:
                vocabulary.append(word)
    vocabulary.append("unk")
    return vocabulary

In [9]:
def preprocess_valid_data(valid_data,vocabulary):
    preprocessed_text=list()
    labels=list()
    for data in valid_data:
        tmp_preprocessed=preprocess(data["reviewText"])
        tmp=[word if word in vocabulary else "unk" for word in tmp_preprocessed]
        preprocessed_text.append(tmp)
        labels.append(data["overall"])
    return preprocessed_text,labels

In [7]:
def embedding_histogram_matrix(preprocessed_text,vocabulary):
    A=list()
    for data in preprocessed_text:
        word_count=np.zeros(len(vocabulary))
        for idx,word in enumerate(vocabulary):
            if word in data:
                word_count[idx]=data.count(word)
        word_count=np.insert(word_count,0,1)
        A.append(word_count)
    A=np.array(A)
    return A

In [15]:
def M2_classifier(A,training_labels):
    b=list()
    for label in training_labels:
        tmp=np.zeros(5)
        if label==1.0:
            tmp[0]=1
        elif label==2.0:
            tmp[1]=1
        elif label==3.0:
            tmp[2]=1
        elif label==4.0:
            tmp[3]=1
        elif label==5.0:
            tmp[4]=1
        b.append(tmp)
    b=np.array(b)
    x_hat=np.linalg.pinv(A)@b
    return x_hat

In [10]:
def accuracy(y_hat,valid_labels):
    dem=0
    for y,label in zip(y_hat,valid_labels):
        newOverall=float(np.argmax(scipy.special.softmax(y))+1)
        #print(newOverall, label)
        if newOverall==label:
            dem+=1
        acc=dem/len(valid_labels)
    return acc
#print(x_hat)

In [11]:
training_data,valid_data=loadData(train_set_path,valid_set_path)

def softmax(x):
    e_x = np.exp(x)
    return e_x / e_x.sum()
y_hat=document@x_hat+assess
dem=0
for y,data in zip(y_hat,valid_data):
    newy=softmax(y)
    newOverall=float(np.argmax(newy)+1)
    #print(newOverall, data["overall"])
    if newOverall==data["overall"]:
        dem+=1
acc=dem/len(training_data)
print(acc)
    

In [12]:
training_preprocessed_text,training_labels=preprocessed_training_data(training_data)
vocabulary=buildGrossary(training_preprocessed_text)
A_training=embedding_histogram_matrix(training_preprocessed_text,vocabulary)

In [13]:
valid_preprocessed_text,valid_labels=preprocess_valid_data(valid_data,vocabulary)
A_valid=embedding_histogram_matrix(valid_preprocessed_text,vocabulary)

In [16]:
x_hat=M2_classifier(A_training,training_labels)
print(x_hat)

[[ 0.06301719  0.01198465  0.05852005  0.11724848  0.74922963]
 [ 0.00799906 -0.00490158 -0.01831092 -0.05979875  0.0750122 ]
 [ 0.18399913  0.03180876 -0.00252505  0.43373081 -0.64701364]
 ...
 [ 0.01106412  1.45520738  1.88683551 -2.14162697 -1.21148004]
 [-0.38061365 -0.08851808  0.20372067  0.85105113 -0.58564007]
 [ 0.          0.          0.          0.          0.        ]]


In [17]:
acc=accuracy(A_valid@x_hat,valid_labels)

In [18]:
print(valid_preprocessed_text[int(random_number)])
print("M2 -",acc)

['work', ',', 'hold', 'save', '.', 'still', 'abl', 'read', 'label', '!', 'would', 'purchas', ',', 'recommend', '.']
M2 - 0.502
